# Tigrigna WhatsApp-Style Keyboard

This notebook creates a Tigrigna keyboard with WhatsApp-style word suggestions that appear as you type!

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import re
import os

## 1. Load the Tigrigna Dictionary

In [2]:
def load_dictionary(file_path='tigrigna_dictionary.txt'):
    """Load the Tigrigna dictionary"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            dictionary = {line.strip() for line in file if line.strip()}
        return dictionary
    except FileNotFoundError:
        print(f"Error: Dictionary file '{file_path}' not found.")
        return set()
    except Exception as e:
        print(f"Error loading dictionary: {e}")
        return set()

# Load the dictionary
dictionary = load_dictionary()
print(f"Loaded {len(dictionary)} words from the dictionary")

Loaded 294 words from the dictionary


## 2. Helper Functions

In [3]:
def levenshtein_distance(s1, s2):
    """Calculate the Levenshtein distance between two strings"""
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

def get_similar_words(word_part, dictionary, max_suggestions=5):
    """Find words that start with or are similar to the given partial word"""
    if not word_part or len(word_part) < 2:
        return []
    
    # Find exact prefix matches first (words that start with the prefix)
    prefix_matches = [word for word in dictionary if word.startswith(word_part)]
    
    # Sort by length to prioritize shorter completions
    prefix_matches.sort(key=len)
    
    # If we have enough prefix matches, return them
    if len(prefix_matches) >= max_suggestions:
        return prefix_matches[:max_suggestions]
    
    # If we need more suggestions, find similar words
    remaining_slots = max_suggestions - len(prefix_matches)
    similar_words = []
    
    for dict_word in dictionary:
        if dict_word not in prefix_matches:  # Skip words we already have
            distance = levenshtein_distance(word_part, dict_word[:len(word_part)])
            if distance <= 2:  # Allow for small differences
                similar_words.append((dict_word, distance))
    
    # Sort by distance and then by length
    similar_words.sort(key=lambda x: (x[1], len(x[0])))
    additional_suggestions = [word for word, _ in similar_words[:remaining_slots]]
    
    # Combine prefix matches with similar words
    return prefix_matches + additional_suggestions

def generate_variants(base_char):
    """Generate vowel variants for a Tigrigna character"""
    if base_char == ' ' or not base_char or len(base_char) != 1:
        return [base_char]
    
    try:
        base_code = ord(base_char)
        variants = []
        for i in range(8):  # Include the 8th form if it exists (base+7)
            try:
                variant_code = base_code + i
                if 0xD800 <= variant_code <= 0xDFFF or variant_code > 0x10FFFF:
                    # Skip invalid Unicode code points
                    continue
                    
                variant = chr(variant_code)
                variants.append(variant)
            except:
                # If this variant doesn't exist, skip it
                pass
        
        return variants if variants else [base_char]
    except:
        return [base_char]

## 3. Create WhatsApp-Style Keyboard with Word Suggestions

In [4]:
def create_whatsapp_keyboard():
    """Create a WhatsApp-style keyboard with word suggestions"""
    # Dark theme styling
    dark_bg = '#212529'      # Main background color
    darker_bg = '#1a1e21'    # Darker background for keys
    text_color = '#e9ecef'   # Text color
    suggestion_bg = '#2b3035'  # Suggestion bar background
    primary_color = '#0dcaf0'  # Accent color (blue)
    
    # Create a chat-like text display area
    chat_display = widgets.HTML(
        value=f'''
        <div style="background-color: {dark_bg}; color: {text_color}; padding: 20px; border-radius: 10px; min-height: 200px; max-height: 300px; overflow-y: auto; margin-bottom: 10px;">
            <div style="text-align: center; background-color: {suggestion_bg}; padding: 15px; border-radius: 8px; margin: 20px auto; max-width: 80%;">
                <p>No messages here yet...</p>
                <p>Send a message or tap the greeting below.</p>
            </div>
        </div>
        ''',
        layout=widgets.Layout(width='100%')
    )
    
    # Create the input area (like WhatsApp's message input)
    text_input = widgets.Text(
        placeholder='Type a message',
        layout=widgets.Layout(
            width='80%', 
            height='40px',
            padding='8px',
            border=f'1px solid {suggestion_bg}',
            border_radius='20px'
        )
    )
    
    # Create send button
    send_button = widgets.Button(
        description='➤',
        button_style='primary',
        layout=widgets.Layout(width='15%', height='40px', border_radius='20px', margin='0 0 0 5px')
    )
    
    # Create input box with text field and send button
    input_box = widgets.HBox(
        [text_input, send_button],
        layout=widgets.Layout(width='100%', padding='10px', align_items='center')
    )
    
    # Create the word suggestion bar (like in WhatsApp)
    suggestion_bar = widgets.HBox(
        [],
        layout=widgets.Layout(
            width='100%',
            padding='5px',
            background_color=suggestion_bg,
            border_radius='5px 5px 0 0',
            display='flex',
            justify_content='space-around'
        )
    )
    
    # Create a container for the vowel variant buttons
    variants_container = widgets.HBox(
        [],
        layout=widgets.Layout(
            width='100%',
            background_color=dark_bg,
            padding='5px',
            display='flex',
            justify_content='space-around'
        )
    )
    
    # Define the keyboard layout - similar to the image
    row1 = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    row2 = ['ቕ', 'ወ', 'ኤ', 'ረ', 'ተ', 'የ', 'ኡ', 'ኢ', 'ኦ', 'ፐ']
    row3 = ['ኣ', 'ሰ', 'ደ', 'ፍ', 'ገ', 'ሀ', 'ጀ', 'ከ', 'ለ']
    row4 = ['ዘ', 'ሸ', 'ጨ', 'ቨ', 'በ', 'ነ', 'መ']
    
    # Create the keyboard container
    keyboard_container = widgets.VBox(
        [],
        layout=widgets.Layout(
            width='100%',
            background_color=dark_bg,
            padding='5px',
            border_radius='0 0 5px 5px'
        )
    )
    
    # Function to create a keyboard button
    def create_key_button(char, width='32px'):
        btn = widgets.Button(
            description=char,
            layout=widgets.Layout(
                width=width,
                height='40px',
                margin='3px',
                border_radius='5px',
                overflow='hidden'
            ),
            style=widgets.ButtonStyle(
                button_color=darker_bg,
                font_weight='bold',
                font_family='Arial, sans-serif',
                text_color=text_color
            )
        )
        return btn
    
    # Function to create a suggestion button
    def create_suggestion_button(word):
        btn = widgets.Button(
            description=word,
            layout=widgets.Layout(
                height='30px',
                margin='2px',
                border_radius='15px',
                overflow='hidden',
                min_width='60px',
                max_width='150px'
            ),
            style=widgets.ButtonStyle(
                button_color=darker_bg,
                font_weight='normal',
                font_family='Arial, sans-serif',
                text_color=text_color
            )
        )
        return btn
    
    # Keep track of current word being typed
    current_word = ['']
    current_base = [None]
    message_history = []
    
    # Function to update word suggestions
    def update_suggestions():
        # Extract the current word being typed
        text = text_input.value
        if not text:
            current_word[0] = ''
            suggestion_bar.children = ()
            return
        
        # Find the current word (at cursor position)
        cursor_pos = text_input.cursor_pos
        text_before_cursor = text[:cursor_pos]
        
        # Extract the current word being typed
        words = re.findall(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF]+', text_before_cursor)
        if words:
            current_word[0] = words[-1]  # Last word before cursor
        else:
            current_word[0] = ''
            suggestion_bar.children = ()
            return
        
        # Generate suggestions based on the current word
        suggestions = get_similar_words(current_word[0], dictionary, max_suggestions=5)
        
        # Create suggestion buttons
        suggestion_buttons = []
        for word in suggestions:
            btn = create_suggestion_button(word)
            
            def on_suggestion_click(b, suggested_word=word):
                # Replace the current word with the suggested word
                current_text = text_input.value
                current_cursor_pos = text_input.cursor_pos
                
                # Find the position of the current word
                current_word_len = len(current_word[0])
                current_word_start = current_cursor_pos - current_word_len
                
                if current_word_start >= 0:
                    # Replace the current word with the suggestion
                    new_text = current_text[:current_word_start] + suggested_word + current_text[current_cursor_pos:]
                    text_input.value = new_text
                    
                    # Move cursor to end of inserted word
                    text_input.cursor_pos = current_word_start + len(suggested_word)
                    
                    # Update suggestions
                    update_suggestions()
            
            btn.on_click(on_suggestion_click)
            suggestion_buttons.append(btn)
        
        suggestion_bar.children = tuple(suggestion_buttons)
    
    # Function to update the chat display
    def update_chat_display():
        chat_html = f'''
        <div style="background-color: {dark_bg}; color: {text_color}; padding: 20px; border-radius: 10px; min-height: 200px; max-height: 300px; overflow-y: auto; margin-bottom: 10px;">
        '''
        
        if not message_history:
            chat_html += f'''
            <div style="text-align: center; background-color: {suggestion_bg}; padding: 15px; border-radius: 8px; margin: 20px auto; max-width: 80%;">
                <p>No messages here yet...</p>
                <p>Send a message or tap the greeting below.</p>
            </div>
            '''
        else:
            for message in message_history:
                chat_html += f'''
                <div style="text-align: right; margin-bottom: 10px;">
                    <div style="display: inline-block; background-color: {primary_color}; color: {darker_bg}; padding: 10px 15px; border-radius: 15px 15px 0 15px; max-width: 80%; text-align: left;">
                        {message}
                    </div>
                </div>
                '''
        
        chat_html += '</div>'
        chat_display.value = chat_html
    
    # Function to send a message
    def send_message(b):
        message = text_input.value.strip()
        if message:
            message_history.append(message)
            text_input.value = ''
            update_chat_display()
            update_suggestions()
    
    # Assign send handler
    send_button.on_click(send_message)
    
    # Function to show vowel variants
    def show_variants(base_char):
        current_base[0] = base_char
        variants = generate_variants(base_char)
        
        # Create variant buttons
        variant_buttons = []
        for variant in variants[:7]:  # Show up to 7 variants
            btn = widgets.Button(
                description=variant,
                layout=widgets.Layout(
                    height='36px',
                    width='36px',
                    margin='2px',
                    border_radius='5px'
                ),
                style=widgets.ButtonStyle(
                    button_color='#198754',  # Green color
                    font_weight='bold',
                    text_color=text_color
                )
            )
            
            def on_variant_click(b, var=variant):
                # Insert the variant at cursor position
                cursor_pos = text_input.cursor_pos
                current_text = text_input.value
                new_text = current_text[:cursor_pos] + var + current_text[cursor_pos:]
                text_input.value = new_text
                text_input.cursor_pos = cursor_pos + len(var)
                update_suggestions()
            
            btn.on_click(on_variant_click)
            variant_buttons.append(btn)
        
        variants_container.children = tuple(variant_buttons)
    
    # Create keyboard rows
    keyboard_rows = []
    
    # Create number row
    number_row = widgets.HBox(
        [create_key_button(char) for char in row1],
        layout=widgets.Layout(justify_content='center')
    )
    keyboard_rows.append(number_row)
    
    # Create character rows
    for row in [row2, row3, row4]:
        row_box = widgets.HBox(
            [create_key_button(char) for char in row],
            layout=widgets.Layout(justify_content='center')
        )
        keyboard_rows.append(row_box)
    
    # Create special row with space, backspace, etc.
    special_row = widgets.HBox(
        [
            create_key_button('?123', width='60px'),  # Special characters toggle
            create_key_button('፡', width='40px'),    # Colon
            create_key_button(' ', width='180px'),  # Space bar
            create_key_button('።', width='40px'),    # Period
            create_key_button('⌫', width='60px')     # Backspace
        ],
        layout=widgets.Layout(justify_content='center')
    )
    keyboard_rows.append(special_row)
    
    # Add language indicator at the bottom
    language_row = widgets.HTML(
        value=f'<div style="text-align: center; color: {text_color}; padding: 5px; font-size: 14px;">< Tigrigna ></div>',
        layout=widgets.Layout(width='100%')
    )
    
    # Add handlers for character buttons
    for row_idx, row_box in enumerate(keyboard_rows):
        for btn in row_box.children:
            char = btn.description
            
            if char == ' ':  # Space bar
                def on_space_click(b):
                    cursor_pos = text_input.cursor_pos
                    current_text = text_input.value
                    new_text = current_text[:cursor_pos] + ' ' + current_text[cursor_pos:]
                    text_input.value = new_text
                    text_input.cursor_pos = cursor_pos + 1
                    current_word[0] = ''
                    suggestion_bar.children = ()
                
                btn.on_click(on_space_click)
            elif char == '⌫':  # Backspace
                def on_backspace_click(b):
                    cursor_pos = text_input.cursor_pos
                    if cursor_pos > 0:
                        current_text = text_input.value
                        new_text = current_text[:cursor_pos-1] + current_text[cursor_pos:]
                        text_input.value = new_text
                        text_input.cursor_pos = cursor_pos - 1
                        update_suggestions()
                
                btn.on_click(on_backspace_click)
            elif char in ['፡', '።']:  # Punctuation
                def on_punctuation_click(b, symbol=char):
                    cursor_pos = text_input.cursor_pos
                    current_text = text_input.value
                    new_text = current_text[:cursor_pos] + symbol + current_text[cursor_pos:]
                    text_input.value = new_text
                    text_input.cursor_pos = cursor_pos + len(symbol)
                    current_word[0] = ''
                    suggestion_bar.children = ()
                
                btn.on_click(lambda b, symbol=char: on_punctuation_click(b, symbol))
            elif char == '?123':  # Toggle special characters - simulated, not functional
                def on_toggle_click(b):
                    # Just a visual indication that the button was clicked
                    b.button_style = 'info'
                    def reset_style():
                        b.button_style = ''
                    import threading
                    threading.Timer(0.3, reset_style).start()
                
                btn.on_click(on_toggle_click)
            else:  # Regular character buttons
                def on_char_click(b, character=char):
                    # Show variants first
                    show_variants(character)
                    
                    # Insert the character
                    cursor_pos = text_input.cursor_pos
                    current_text = text_input.value
                    new_text = current_text[:cursor_pos] + character + current_text[cursor_pos:]
                    text_input.value = new_text
                    text_input.cursor_pos = cursor_pos + len(character)
                    
                    # Update suggestions
                    update_suggestions()
                
                btn.on_click(lambda b, character=char: on_char_click(b, character))
    
    # Add observer for text input changes
    def on_text_change(change):
        if change['name'] == 'value':
            update_suggestions()
            
            # Check for Enter key (newline character)
            if change['new'] and '\n' in change['new']:
                # Remove the newline character
                text_input.value = change['new'].replace('\n', '')
                # Send the message
                send_message(None)
    
    text_input.observe(on_text_change, names='value')
    
    # Assemble all components
    keyboard_container.children = [variants_container] + keyboard_rows + [language_row]
    
    # Add some initial suggested greetings like WhatsApp
    greeting_suggestions = ['ሰላም', 'ጽቡቕ', 'ከመይ ኣለኻ', 'ሃሎ']
    greeting_buttons = []
    
    for greeting in greeting_suggestions:
        btn = create_suggestion_button(greeting)
        
        def on_greeting_click(b, text=greeting):
            text_input.value = text
            update_suggestions()
        
        btn.on_click(lambda b, text=greeting: on_greeting_click(b, text))
        greeting_buttons.append(btn)
    
    suggestion_bar.children = tuple(greeting_buttons)
    
    # Create the complete UI
    complete_ui = widgets.VBox([
        widgets.HTML('<h2 style="color: #e9ecef; text-align: center;">ትግርኛ - Tigrigna Chat</h2>'),
        chat_display,
        suggestion_bar,
        input_box,
        keyboard_container
    ], layout=widgets.Layout(width='100%', background_color=dark_bg, padding='15px'))
    
    return complete_ui

## 4. Create and Display the WhatsApp-Style Keyboard

In [5]:
# Create and display the WhatsApp-style keyboard
whatsapp_keyboard = create_whatsapp_keyboard()
display(whatsapp_keyboard)